In [1]:
import polars as pl

In [24]:
movie_data = pl.read_parquet('data/processed/movie_data.parquet').with_columns(
    release_year = pl.concat_str(pl.col('release_date').dt.year(),pl.lit('-01-01'))
).with_columns(
    release_year = pl.col('release_year').str.to_date()
)

In [42]:
summarized = movie_data.group_by('release_year').agg(
    pl.col('sequel').sum().alias('sequels'),
    pl.col('remake').sum().alias('remakes'),
    pl.len().alias('total')
    ).with_columns(
        (100 * pl.col('sequels')/pl.col('total')).alias('sql_pct'),
        (100 * pl.col('remakes')/pl.col('total')).alias('remake_pct')
    ).sort('release_year').filter(pl.col('release_year').is_not_null())
summarized.write_json('data/processed/summarized.json')

In [50]:
summarizedPD = summarized.to_pandas()
summarizedPD.head()

,release_year,sequels,remakes,total,sql_pct,remake_pct
0,1970-01-01,6,1,184,3.260870,0.543478
1,1971-01-01,6,1,191,3.141361,0.523560
2,1972-01-01,9,1,137,6.569343,0.729927
3,1973-01-01,7,3,200,3.500000,1.500000
4,1974-01-01,11,0,178,6.179775,0.000000


In [54]:
alt.Chart(summarizedPD).mark_bar().encode(
    x='release_year',
    y='total'
)

alt.Chart(...)

In [66]:
x = alt.Chart(summarizedPD).mark_bar().encode(
    x='release_year:T',
    y='sql_pct:Q'
# ).configure_mark(fill='green')
)
y = alt.Chart(summarizedPD).mark_bar().encode(
    x='release_year:T',
    y='remake_pct:Q'
)

alt.LayerChart(...)